## mongodb

In [ ]:
!pip install pymongo[srv]==3.10.1

In [ ]:
from pymongo import MongoClient

# client = MongoClient('localhost', 27017)
client = MongoClient('string')

In [ ]:
db = client.pymongo_test

In [ ]:
# ver bases de datos
for db_info in client.list_database_names():
    print(db_info)

In [ ]:
posts = db.posts
data = {
    "title": "mongodb2",
    "content": "some content2",
    "author": "MongoDB2"
}
result = posts.insert_one(data)
print(result.inserted_id)


In [ ]:
posts = db.posts
data_1 = {
    "title": "mongodb3",
    "content": "some content3",
    "author": "MongoDB3",
    "courses": []
}
data_2 = {
    "title": "mongodb4",
    "content": "some content4",
    "author": "MongoDB4"
}
data_3 = {
    "title": "mongodb5",
    "content": "some content4",
    "author": "MongoDB4"
}
result = posts.insert_many([data_1, data_2, data_3])
print(result.inserted_ids)

In [ ]:
posts = db.posts

post_found = posts.find_one({"author": "MongoDB4"})
print(post_found)
print("-------")

posts_found = posts.find({"author": "MongoDB4"})
for x in posts_found:
    print(x)

In [ ]:
posts = db.posts

posts_found = posts.find({"title": { "$regex": "^mongo" }})
for x in posts_found:
    print(x)

In [ ]:
!pip install mongoengine

In [ ]:
from mongoengine import *

connect(host = 'string')

In [ ]:
import datetime

class Post(Document):
    title = StringField(required=True, max_length=200)
    content = StringField(required=True) 
    author = StringField(required=True, max_length=100)
    published = DateTimeField(default=datetime.datetime.now())


In [ ]:
post_1 = Post(
    title="MongoEngine 1",
    content="MongoEngine 1",
    author="MongoEngine"
)
post_1.save()
print(post_1.id)

In [ ]:
# mongoimport --db sample_algo --collection data --file ./.../data.json

In [ ]:
from pymongo import MongoClient

client = MongoClient('string')

In [ ]:
db = client.sample_mflix
movies = db.movies

# view all
#results = users.find()
#for x in results:
#    print(x)

movies.find().count()

In [ ]:
# view only some columns
results = movies.find({}, {"title": 1, "poster": 1, "_id": 0})
for x in results:
    print(x)

In [ ]:
results = movies.find({"rated": "TV-G", "year": {"$gt": 1904}}).limit(5)
for x in results:
    print(x)

In [ ]:
results = movies.find({"plot": { "$regex": "^A " }, "num_mflix_comments": {"$gt": 0}})
for x in results:
    print(x)

## scraping

In [ ]:
!pip install requests beautifulsoup4

In [ ]:
# https://cuantoestaeldolar.pe/ # tipo de cambio SUNAT

import requests

raw = requests.get("https://cuantoestaeldolar.pe/")
print(raw.status_code)
print(raw.text)


In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(raw.text, 'html.parser')
data = soup.find_all("p", class_="ValueCurrency_item_cost__Eb_37")
tipo_de_cambio = {
    "sunat": {
        "compra": float(data[0].text),
        "venta": float(data[1].text),
    },
    "paralelo": {
        "compra": float(data[2].text),
        "venta": float(data[3].text),
    }
}
print(tipo_de_cambio)

## apis

In [ ]:
# https://github.com/public-apis/public-apis

import requests

x = requests.get("https://ipinfo.io")
print(x.status_code)
print(x.text)
print("-----")
print(x.json())

data = x.json()
print("based on your ip, you are in {}".format(data["city"]))

In [ ]:
# https://pokeapi.co/

def get_pokemon_data(pokemon_name):
    raw = requests.get("https://pokeapi.co/api/v2/pokemon/{}".format(pokemon_name))
    data = raw.json()
    return {
        "weight": data["weight"],
        "height": data["height"],
        "name": data["name"],
        "type": [x["type"]["name"] for x in data["types"]],
    }

get_pokemon_data("pikachu")

In [ ]:
type_search = "electric"

raw = requests.get("https://pokeapi.co/api/v2/pokemon?limit=100&offset=0")
data = raw.json()
for x in data["results"]:
    raw2 = requests.get(x["url"])
    data2 = raw2.json()
    data2_types = [x["type"]["name"] for x in data2["types"]]
    if type_search in data2_types:
        print(x["name"])

## scraper

In [ ]:
raw = requests.get("https://www.imdb.com/es-es/list/ls024149810/", headers={
    "Host": "www.imdb.com",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:140.0) Gecko/20100101 Firefox/140.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "es-ES,es;q=0.8,en-US;q=0.5,en;q=0.3",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "DNT": "1",
    "Sec-GPC": "1",
    "Alt-Used": "www.imdb.com",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
})
print(raw.status_code)
print(raw.text)

In [ ]:
soup = BeautifulSoup(raw.text, "html.parser")
for x in soup.find_all("li", class_="ipc-metadata-list-summary-item")[:25]:
    print(x.find("h3", class_="ipc-title__text").text)
    for y in x.find_all("a", class_="sttd-director-item"):
        print(y.text)

In [ ]:
soup = BeautifulSoup(raw.text, "html.parser")
scores = []
all_views = 0
for x in soup.find_all("li", class_="ipc-metadata-list-summary-item")[:25]:
    scores.append(float(x.find("span", class_="metacritic-score-box").get_text()))
    views_raw = str(x.find("span", class_="ipc-rating-star--voteCount").get_text()[2:-1])
    views_raw_number, views_raw_indicator = views_raw.split("\xa0")
    if views_raw_indicator == "M":
        all_views = all_views + float(views_raw_number.replace(",", ".")) * 1000000
    if views_raw_indicator == "mil":
        all_views = all_views + float(views_raw_number.replace(",", ".")) * 1000

print("average punctuation based on meta: {}".format(sum(scores)/len(scores)))
print("all views is {}".format(all_views))

## scraping - 
## pagina web - machine learning - predecir CV - mejor empleo
## mejores empleos

In [ ]:
import requests
import time

# time.sleep(30)

raw_data = requests.post("https://www.bumeran.com.pe/api/avisos/searchV2?pageSize=100&page=0&sort=RECIENTES", headers={
    "Host": "www.bumeran.com.pe",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:140.0) Gecko/20100101 Firefox/140.0",
    "Accept": "application/json",
    "Accept-Language": "es-ES,es;q=0.8,en-US;q=0.5,en;q=0.3",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Content-Type": "application/json",
    "x-site-id": "BMPE",
    "Origin": "https://www.bumeran.com.pe",
    "DNT": "1",
    "Connection": "keep-alive",
    "Referer": "https://www.bumeran.com.pe/empleos.html",
    "TE": "trailers",
})
print(raw_data.status_code)
print(raw_data.text)

201
{"number":0,"size":100,"total":30429,"content":[{"id":1117728802,"titulo":"ASISTENTE DE ATRACCIÓN DEL TALENTO / CUSCO / REF 676 2025","detalle":"Responsabilidades Ejecutar los programas diseñados para el desarrollo del talento humano de los trabajadores en las regiones a su cargo.  Coordinar la programación de las contrataciones de las regiones asignadas.  Ejecutar los procesos de reclutamiento y atracción del talento de las regiones asignadas y áreas usuarias. Gestionar y controlar a través de la plataforma virtual los procesos de atracción del talento de las regiones a su cargo de acuerdo a los plazos establecidos para cumplir con los objetivos del negocio y la contratación del personal nuevo, así como mantener debidamente actualizada y ordenada la documentación de sustento de los procesos.  Realizar la solicitud, recepción y revisión de documentación de los ingresos de nuevo personal conforme lo establecido en los procedimientos internos.  Coordinar la elaboración de contratos d

In [7]:
data = raw_data.json()
print(data)

{'number': 0, 'size': 100, 'total': 30429, 'content': [{'id': 1117728802, 'titulo': 'ASISTENTE DE ATRACCIÓN DEL TALENTO / CUSCO / REF 676 2025', 'detalle': 'Responsabilidades Ejecutar los programas diseñados para el desarrollo del talento humano de los trabajadores en las regiones a su cargo.  Coordinar la programación de las contrataciones de las regiones asignadas.  Ejecutar los procesos de reclutamiento y atracción del talento de las regiones asignadas y áreas usuarias. Gestionar y controlar a través de la plataforma virtual los procesos de atracción del talento de las regiones a su cargo de acuerdo a los plazos establecidos para cumplir con los objetivos del negocio y la contratación del personal nuevo, así como mantener debidamente actualizada y ordenada la documentación de sustento de los procesos.  Realizar la solicitud, recepción y revisión de documentación de los ingresos de nuevo personal conforme lo establecido en los procedimientos internos.  Coordinar la elaboración de con

In [9]:
!pip install python-slugify

In [11]:
from slugify import slugify

titulo = "Funcionario de Negocios / Hibrido"
empresa = " ADN SELECTION S.A.C."
id = 1117728766

# https://www.bumeran.com.pe/empleos/funcionario-de-negocios-hibrido-adn-selection-s.a.c.-1117728766.html

url = "https://www.bumeran.com.pe/empleos/{}-{}-{}.html".format(slugify(titulo), slugify(empresa), id)
print(url)

https://www.bumeran.com.pe/empleos/funcionario-de-negocios-hibrido-adn-selection-s-a-c-1117728766.html


In [ ]:
import requests
import time

results = []
for i in range(1, 6):
    raw_data = requests.post("https://www.bumeran.com.pe/api/avisos/searchV2?pageSize=100&page={}&sort=RECIENTES".format(i - 1), headers={
        "Host": "www.bumeran.com.pe",
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:140.0) Gecko/20100101 Firefox/140.0",
        "Accept": "application/json",
        "Accept-Language": "es-ES,es;q=0.8,en-US;q=0.5,en;q=0.3",
        "Accept-Encoding": "gzip, deflate, br, zstd",
        "Content-Type": "application/json",
        "x-site-id": "BMPE",
        "Origin": "https://www.bumeran.com.pe",
        "DNT": "1",
        "Connection": "keep-alive",
        "Referer": "https://www.bumeran.com.pe/empleos.html",
        "TE": "trailers",
    })
    if raw_data.ok:
        results.extend(raw_data.json()["content"])
    time.sleep(5)

print(len(results))

1
2
3
4
5
500


In [17]:
import csv
from slugify import slugify
from datetime import datetime

def clean_text(text):
    return text.lower().replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u").replace(".", "").replace(",", "")

with open("./03/jobs.csv", "w") as csv_file:
    writer = csv.DictWriter(
        csv_file,
        fieldnames = [
            "FECHA_SCRAPING",
            "EMPRESA",
            "PUESTO",
            "PUESTO_LIMPIO",
            "DESCRIPCION",
            "DESCRIPCION_LIMPIO",
            "URL",
        ]
    )
    writer.writeheader()
    for row in results:
        writer.writerow({
            "FECHA_SCRAPING": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "EMPRESA": row["empresa"],
            "PUESTO": row["titulo"],
            "PUESTO_LIMPIO": clean_text(row["titulo"]),
            "DESCRIPCION": row["detalle"],
            "DESCRIPCION_LIMPIO": clean_text(row["detalle"]),
            "URL": "https://www.bumeran.com.pe/empleos/{}-{}-{}.html".format(slugify(row["titulo"]), slugify(row["empresa"]), row["id"])
        })


In [ ]:
# usando pickle

import pickle

pickle.dump(results, open("./03/jobs.pickle", "wb"), protocol=pickle.HIGHEST_PROTOCOL)